### 3D Reconstruction (done by Michael Hafner)

After the extended lines of the object got determined it is possible now to calculate the intersections of those lines which provide the vanishing points.

With this information an algorithm from https://kusemanohar.wordpress.com/2014/03/18/3d-reconstruction-with-single-view/ got used to determine the projection matrix. With that matrix it should be possible to get the projection of each plane. As you can see in the following image, the algorithm failed.

In [2]:
from IPython.display import Image
Image(url= "report-images/failed-transformation.png")

Since it was not possible to project the planes automatically using the above mentioned algorithm and finding or come up with another solution, a manual approach got used.
With it first all side lengths of the object need to get measured to determine the ratios of the edge. After that the position (x-y values in the image) of every vertex got detected. The following code shows the transformation from the camera view to the plane view.

In [ ]:
trans = image.copy()
#source coordinates (edges in the image)
src = np.array([165, 293,
                413, 161,
                351, 423,
                195, 557, ]).reshape((4, 2))
#destination coordinates (rectangular shape of the corresponding plane, manually calculated from the lengths)
dst = np.array([165, 161,
                355, 161,
                355, 427,
                165, 427]).reshape((4, 2))
#using skimage’s transform module where ‘projective’ is our desired parameter
tform = transform.estimate_transform('projective', src, dst)
tf_img = transform.warp(trans, tform.inverse)

yz_plane = tf_img[161:427, 165:355]

plt.imshow(yz_plane)
plt.axis(False)
cv2.imwrite('object_images/yz_plane.png', 255 * yz_plane)

The source coordinates (src) got determined as previously mentioned of the vertices of each plane. To become now the plane view, the destination coordinates of those vertices need to be calculated. This was done by creating a rectangle using the vertex coordinates and the ratios of the objects edges. After that those parameter got used to transform the image and cut off the surrounding unnecessary and distorted pixel values.

In [3]:
from IPython.display import Image
Image(url= "report-images/man-transformation.png")

This transformation can lead to distortions if there is another object on the corresponding plane. In the example below, there is an ear plug on top of the book cover. It can be seen, that the plug got wrapped by the transformation. So the transformation of object planes just works well on plain surfaces without other objects on top of it or other view restrictions (e.g. chimney on a roof)

In [4]:
from IPython.display import Image
Image(url= "report-images/distortion-plug.png")

After the transformation, the numpy-stl library got used to create a stl file, which later can be used for a 3D suite e.g. Blender to process the created 3D object further (map the image planes to the object). The code bellow shows that first the vertices of the object need to be created. This was done by manually defining vertices corresponding to the edge length of the real object. After that, the vertices need to get connected to form a surface, this was done by using triangles. Therefore, the faces array contains the 3 numbers of the vertices which form the face. At last a 3D mesh got created and stored to the stl-file.

In [ ]:
# Define the 8 vertices of the cube, corresponding to the ratios of the real object
vertices = np.array([
    [-1, -1, -1],
    [+55, -1, -1],
    [+55, +190, -1],
    [-1, +190, -1],
    [-1, -1, +266],
    [+55, -1, +266],
    [+55, +190, +266],
    [-1, +190, +266]])
# Define the 12 triangles composing the cube
faces = np.array([
    [0, 3, 1],
    [1, 3, 2],
    [0, 4, 7],
    [0, 7, 3],
    [4, 5, 6],
    [4, 6, 7],
    [5, 1, 2],
    [5, 2, 6],
    [2, 3, 6],
    [3, 7, 6],
    [0, 1, 5],
    [0, 5, 4]])

# Create the mesh
cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        cube.vectors[i][j] = vertices[f[j], :]

# Write the mesh to file "cube.stl"
cube.save('cube.stl')

This stl-file now can be opened with blender. To map the previous determined object planes to the corresponding images, the following steps are necessary:
1. Open the file in Blender

In [5]:
from IPython.display import Image
Image(url= "report-images/blender-plain.png")

2. Create a material for each object plane
3. Select a face and assign it to the right material
4. Change the **Base Color** to the image of the surface

In [6]:
from IPython.display import Image
Image(url= "report-images/blender-assign.png")

5. It is then possible to adjust the image (since the object ratios match to the image ratios, no further adjustment was needed)

In [7]:
from IPython.display import Image
Image(url= "report-images/blender-mapped.png")

### Conclusion of the Classic Approach

In the recent approach it was found out that it is possible to create a 3D object using just a single input image. However, the process cannot be fully automated and the user needs to specify additional lines, the object ratios and angles manually. The image transformation to object planes also has limitations, as further objects in the scene can distort the image, and the view/angle is crucial for obtaining the desired result.

The success of the process is highly dependent on the complexity of the object being reconstructed. It was time-consuming and will definitely become more challenging for reconstructing complex objects. So the provided source code in the Classic-Approach/ folder just works for the given book image. For other objects the code or rather the points and ratios need to be recalculated and measured.

Furthermore, the presented approach is suitable for simple objects, more complex objects would require alternative techniques like ToF or structured light to enhance accuracy and performance.